In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
from torch.autograd.functional import jacobian
from operators import Δ, div, D, mdotb, bdotm, mdotm, bdotb
from torch.nn.functional import relu, max_pool2d, avg_pool2d, dropout, dropout2d, interpolate

In [2]:
class Complex_ReLU(nn.Module):
    def forward(self, input):
        return relu(input.real).type(torch.complex64)+1j*relu(input.imag).type(torch.complex64)

In [14]:
complex_net = nn.Sequential(
            nn.Linear(1, 4).to(torch.cfloat),
            Complex_ReLU(),
            nn.Linear(4, 1).to(torch.cfloat)
        )
tensor = torch.randn(4, 2).requires_grad_()
tensor
torch.view_as_complex(tensor).unsqueeze(1)

tensor([[-0.1237+0.7677j],
        [-0.8950-1.2208j],
        [-0.2374-1.4870j],
        [ 0.8238+0.1815j]], grad_fn=<UnsqueezeBackward0>)

In [15]:
u = complex_net(torch.view_as_complex(tensor).unsqueeze(1))
u = torch.exp(torch.abs(1j*u))

In [16]:
div(u, tensor)

/home/frederik/anaconda3/envs/dgm_env/lib/python3.8/site-packages/torch/autograd/__init__.py:223: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  return Variable._execution_engine.run_backward(


tensor([[-0.0384,  0.0527],
        [ 0.1280, -0.1532],
        [-0.0318, -0.1729],
        [-0.0388,  0.0143]], grad_fn=<AddBackward0>)

In [4]:
var = torch.tensor([[1.0, 1.0],
                   [4.0, 3.0]], requires_grad=True)
t = torch.tensor([[2.0], 
                  [3.0]], requires_grad=True)

var_2 = torch.tensor([[1.0],
                   [3.0]], requires_grad=True)
t_2 = torch.tensor([[1.0], 
                  [2.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0].unsqueeze(1), 3) + xt[:,-1].unsqueeze(1)

In [ ]:
spatial_net = nn.Sequential(
            nn.Linear(2, 4),
            nn.ELU(),
            nn.Linear(4, 4),
            nn.ELU(),
            nn.Linear(4, 1)
        )

In [ ]:
out = u(var, t)
out_2 = u(var_2, t_2)

In [ ]:
torch.sum(jacobian(spatial_net, var), dim=0)

In [ ]:
out = spatial_net(var)
div(out, var)

In [ ]:
print(div(out.unsqueeze(1), t))
print(div(out_2.unsqueeze(1), t_2))

In [ ]:
print(div(out.unsqueeze(1), var))
print(div(out_2.unsqueeze(1), var_2))

In [ ]:
print(Δ(out.unsqueeze(1), var))
print(Δ(out_2.unsqueeze(1), var_2))

In [ ]:
loss = torch.square(div(out, t) - Δ(out, var)).mean()

In [ ]:
loss.backward()

In [24]:
var_1 = torch.tensor([[1.0],
                      [4.0]], requires_grad=True)
var_2 = torch.tensor([[2.0],
                      [8.0]], requires_grad=True)
var = [var_1, var_2]
C = torch.tensor([[2.0, 4.0],
                  [1.0, 3.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0].unsqueeze(1), 3) + torch.pow(xt[:,-1].unsqueeze(1), 2)

In [25]:
us = u(var_1, var_2)
print(us)

tensor([[ 4.1667],
        [74.6667]], grad_fn=<AddBackward0>)


In [29]:
D(us, var_1)

tensor([[0.5000],
        [8.0000]], grad_fn=<CatBackward>)

In [4]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)

In [5]:
def m_func(x, y):
    if isinstance(x, list):
        x = torch.cat(x, dim=-1)   
        return torch.einsum("bi, ij -> bj", x, y)
    y = torch.cat(y, dim=-1)
    return torch.einsum("ij, bj -> bi", x, y)
        
def v_func(x, y):
    if isinstance(x, list):
        x = torch.cat(x, dim=-1)
    if isinstance(y, list):
        y = torch.cat(y, dim=-1)
    
    return torch.einsum("bi, bi -> b", x, y).unsqueeze(1)
    
        

In [21]:
var_1 = torch.tensor([[1.0],
                      [4.0],
                      [4.0]], requires_grad=True)
var_2 = torch.tensor([[2.0],
                      [8.0],
                      [3.0]], requires_grad=True)
var_3 = torch.tensor([[1.0],
                      [1.0],
                      [1.0]], requires_grad=True)
var = [var_1, var_2, var_3]
C = torch.tensor([[2.0, 4.0],
                  [1.0, 3.0]], requires_grad=True)

In [22]:
(C @ torch.cat(var[:2], dim=-1).T).T

tensor([[10.,  7.],
        [40., 28.],
        [20., 13.]], grad_fn=<PermuteBackward>)

In [23]:
(C |mdotb| torch.cat(var[:2], dim=-1))

tensor([[10.,  7.],
        [40., 28.],
        [20., 13.]], grad_fn=<ViewBackward>)

In [24]:
torch.cat((var_1, var_2, var_1), dim=-1)

tensor([[1., 2., 1.],
        [4., 8., 4.],
        [4., 3., 4.]], grad_fn=<CatBackward>)